IMPORTATION DES OUTILS ET PARAMETRES

In [86]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive
from IPython.display import display, clear_output
import traitlets
import numpy as np

pd.set_option('display.max_rows', 500)

style = {'description_width': 'initial'}

df_loyer = pd.read_excel("data/loyer_par_metre_carre.xlsx")
df_loyer['Code'] = df_loyer['Code'].astype(str)

df_achat = pd.read_csv("data/communesdvf2024.csv")
df_achat['INSEE_COM'] = df_achat['INSEE_COM'].astype(str)


PARTIE NETTOYAGE DES DONNES

In [87]:
condition_nettoyage = df_loyer["Loyer d'annonce"] == 'N/A - résultat non disponible'
indices_supp = df_loyer[condition_nettoyage].index
df_loyer = df_loyer.drop(indices_supp)

##Ajout colonne code département
df_loyer['code_dept'] = df_loyer['Code'].astype(str).str.slice(0, 2)
condition_nettoyage = (df_loyer['code_dept'] == '97') | (df_loyer['code_dept'] == '2A') | (df_loyer['code_dept'] == '2B')
indices_supp = df_loyer[condition_nettoyage].index
df_loyer = df_loyer.drop(indices_supp) #Supression des outre-mer et de la corse pour simplifier

##Ajout colonne code département
df_achat['code_dept'] = df_achat['INSEE_COM'].astype(str).str.slice(0, 2)
condition_nettoyage = (df_achat['code_dept'] == '97') | (df_achat['code_dept'] == '2A') | (df_achat['code_dept'] == '2B')
indices_supp = df_achat[condition_nettoyage].index
df_achat = df_achat.drop(indices_supp) #Supression des outre-mer et de la corse pour simplifier

TRAITEMENT

Varibale global parametre_entree_utilisateur qui va permettre de savoir quel traitement appliquer par la suite
Ajout d'un observateur pour recalculer quand on la modifie

In [88]:
class observation_parametre (traitlets.HasTraits):
    parametres_entree_utilisateur = traitlets.Dict()

Traitement si zone = 'France'

In [ ]:
def on_parametres_change(change):
    
    if change['new']['zone'] == 'France':

        loyers_moyens_par_dept = df_loyer.groupby('code_dept')["Loyer d'annonce"].mean()
        df_dept = loyers_moyens_par_dept.reset_index()
        df_dept.columns = ['Numéro de département', 'Loyer moyen']

        prix_moyens_par_dept = df_achat.groupby('code_dept')["Prixm2Moyen"].mean()
        
        df_dept['Prix moyen'] = df_dept['Numéro de département'].map(prix_moyens_par_dept)

        df_dept['Rentabilité'] = ((df_dept['Loyer moyen']*12)/df_dept['Prix moyen'])*100

        ##Filtre des outliers car quelques valeurs aberantes

        Q1 = df_dept['Rentabilité'].quantile(0.25)
        Q3 = df_dept['Rentabilité'].quantile(0.75)
        IQR = Q3 - Q1
        lim_inf = Q1 - 1.5 * IQR
        lim_sup = Q3 + 1.5 * IQR

        df_dept_propre = df_dept[(df_dept['Rentabilité'] >= lim_inf) & (df_dept['Rentabilité'] <= lim_sup)]
 
        geo_dept = gpd.read_file("data/departements.geojson")
        geo_dept['code'] = geo_dept['code'].astype(str)

        carte_df = geo_dept.merge(
            df_dept_propre, 
            left_on='code', 
            right_on='Numéro de département', 
            how='left'
        )

        carte_df['Loyer moyen'] = pd.to_numeric(carte_df['Loyer moyen'])
        
        carte_df['Rentabilité'] = carte_df['Rentabilité'].replace([np.inf, -np.inf], np.nan)

        fig, ax = plt.subplots(1, 1, figsize=(10, 10))

        carte_df.plot(
            ax=ax,
            color="lightgrey",
            edgecolor="0.8",
            linewidth=0.5
        )

        carte_avec_donnees = carte_df.dropna(subset=['Rentabilité'])

        carte_avec_donnees.plot(
            column='Rentabilité', 
            cmap='Reds',           
            linewidth=0.5,         
            ax=ax,                 
            edgecolor='0.8',       
            legend=True,
            legend_kwds={
                'label': "Rentabilité de location (%)", # C'est 'title' qu'il faut ici
                'orientation': "horizontal",
                'shrink': 0.7 
            }
        )
        
        ax.set_axis_off() 
        ax.set_title(
        "Rentabilité moyenne des départements en France", 
        fontdict={'fontsize': '16', 'fontweight' : '3'}
        ) 

        plt.show()
        
        
        
#Création de l'instance pour appliquer l'observation
etat_app = observation_parametre()
etat_app.observe(on_parametres_change, names='parametres_entree_utilisateur')

Entrée utilisateurs

In [ ]:
def filtre(code, zone):

    etat_app.parametres_entree_utilisateur = {'zone': zone, 'code': code}

    if (zone == "France"):
        return "Traitement sur la France"
    
    ## Partie département
    if (zone == "Département"):
        if (code == ""):
            return 'Veuillez entrer un numéro de département'
        
        if (not code.isdigit()):
             return 'Non valide (ex: 91)'
        
        num_dept = int(code)
        if (num_dept < 1 or num_dept > 95 or len(code) < 2):
            return 'Département non valide'
        else:
            ligne = df_loyer[df_loyer["Code"].str.startswith(code)]
            return ligne
    
    ## Partie Commune
    if (zone == "Commune"):
        if (code == ""):
            return 'Veuillez entrer un numéro de code postal'
            
        if (not code.isdigit()):
            return 'Non valide (ex: 91120)'

        if (len(code) < 5):
            return 'Veuillez entrer plus de chiffres'
        else :
            ligne = df_loyer[df_loyer["Code"].str.startswith(code)]
            return ligne

selecteur_zone = widgets.RadioButtons(options=['France', 'Département', 'Commune'], value='France', description='Zone de recherche:', style=style)

output_dynamique = widgets.Output()

def on_zone_change(change):
    zone_choisie = change['new']
    
    with output_dynamique:
        clear_output(wait=True)
        
        match zone_choisie:
            
            case "France":
                print(filtre(code=None, zone="France"))
            
            case "Département":
                code_dept_input = widgets.Text(value='', description='Code Département:', style=style)
                widgets.interact(filtre, code=code_dept_input, zone=widgets.fixed("Département"))

            
            case "Commune":
                code_insee_input = widgets.Text(value='', description='Code INSEE:', style=style)
                widgets.interact(filtre, code=code_insee_input, zone=widgets.fixed("Commune"))

selecteur_zone.observe(on_zone_change, names='value')

display(selecteur_zone, output_dynamique)

on_zone_change({'new': selecteur_zone.value})

RadioButtons(description='Zone de recherche:', options=('France', 'Département', 'Commune'), style=Description…

Output()

CODE DE COTE:

#Slide bar du nombre de mètres carrés#

nb_metre_carre = widgets.IntSlider(value=9, min=9, max=500, description='Nombre de métres carré:')
display(nb_metre_carre)